In [1]:
from ioMicro import *

In [10]:
fl =r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\H1_MER_set1\Conv_zscan__030.zarr'
im = read_im(fl)
import napari
napari.view_image(im);

In [1]:
from ioMicro import *
fls_all = glob.glob(r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\H1_MER_set*\*.zarr')

In [13]:
#len(fls_all)

In [42]:
save_folder =r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis'
if True:
    fl = r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\H1_MER_set1\Conv_zscan__145.zarr'
    new_segmentation(fl =fl,
                             psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=1500,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=True,plt_val=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  7.99it/s]


In [8]:
save_folder =r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis'
for fl in tqdm(fls_all):
    try:
        new_segmentation(fl =fl,
                              psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=1500,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=True,plt_val=False)
    except:
        print("Failed",fl)

In [9]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')

p1,p99=0,np.percentile(imd[::3,::4,::4],97)#8000
img = np.array(np.clip((imd[::3,::4,::4]-p1)/(p99-p1),0,1),dtype=np.float32)
masks, flows, styles, diams = model.eval(img,z_axis=0, diameter=15, channels=[0,0],
                                         flow_threshold=-10,cellprob_threshold=-10,normalize=False,
                                         do_3D=False,stitch_threshold=0.5,
                                         progress=True)

In [40]:
v = napari.view_image(img)
v.add_labels(masks)

<Labels layer 'masks' at 0x1a9ebcddc70>

In [14]:
np.percentile(im_dapi_[::3,::4,::4],99)

2423.224697265623

In [10]:
napari.view_image(im_dapi_)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.17575000000000002, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(18.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(18, 1500, 1500), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_dapi_' at 0x1a6beb58bb0>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPos

In [4]:
psf = np.load('\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy')

In [39]:
imd = full_deconv(im_dapi,s_=500,pad=100,psf=psf,parameters={'method': 'wiener', 'beta': 0.01},gpu=True,force=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  8.07it/s]


In [40]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')


In [46]:
np.percentile(imd,97)

12677.7783203125

In [49]:
v = napari.view_image(img)
v.add_labels(masks)

<Labels layer 'masks' at 0x1e619b98790>

In [111]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')
masksA = []
for img in tqdm(imd[::3,::4,::4]):
    p1 = 0#np.percentile(img,1)
    #if p99 is None:
    p99 = 23000#np.percentile(img,99.)
    img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
    masks, flows, styles, diams = model.eval(img, diameter=15, channels=[0,0],
                                         flow_threshold=1,cellprob_threshold=-10,normalize=False)
    masksA.append(masks)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [01:34<00:00,  7.23s/it]


In [112]:
masksA = np.array(masksA)

In [113]:
v = napari.view_image(imd[::3,::4,::4],contrast_limits=[p1,p99])
v.add_labels(masksA)

<Labels layer 'masksA' at 0x240c553ac70>

In [28]:
import napari
v = napari.view_image(imd)
v.add_image(im_dapi)

<Image layer 'im_dapi' at 0x23e3486bfd0>

In [17]:
napari.view_image(im_dapi);

In [ ]:
psf=None,
    plt_val=False,
    parameters={'method': 'wiener', 'beta': 0.001, 'niter': 50},
    gpu=False,
    force=False,
    pad=None,

In [22]:
def full_deconv(im_,s_=500,pad=100,psf=None,parameters={'method': 'wiener', 'beta': 0.001, 'niter': 50},gpu=True,force=False):
    im0=np.zeros_like(im_)
    sx,sy = im_.shape[1:]
    ixys = []
    for ix in np.arange(0,sx,s_):
        for iy in np.arange(0,sy,s_):
            ixys.append([ix,iy])
    
    for ix,iy in tqdm(ixys):#ixys:#tqdm(ixys):
        imsm = im_[:,ix:ix+pad+s_,iy:iy+pad+s_]
        imt = apply_deconv(imsm,psf=psf,parameters=parameters,gpu=gpu,plt_val=False,force=force)
        start_x = ix+pad//2 if ix>0 else 0
        end_x = ix+pad//2+s_
        start_y = iy+pad//2 if iy>0 else 0
        end_y = iy+pad//2+s_
        #print(start_x,end_x,start_y,end_y)
        im0[:,start_x:end_x,start_y:end_y] = imt[:,(start_x-ix):(end_x-ix),(start_y-iy):(end_y-iy)]
    return im0

In [14]:
#[ln for ln in open(r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\H1_MER_set2\Conv_zscan__010.power','r')]

In [15]:
#np.array(im[9])

In [ ]:
"""Using cellpose with nuclei mode"""
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=use_gpu, model_type=model)
#decided that resampling to the 4-2-2 will make it faster
#im_dapi_3d = im_dapi[::rescz,::resc,::resc].astype(np.float32)
chan = [0,0]
masks_all = []
flows_all = []
from tqdm import tqdm
for im in tqdm(im_dapi):
    im_ = np.array(im,dtype=np.float32)
    img = (cv2.blur(im_,(2,2))-cv2.blur(im_,(150,150)))[::resc,::resc]
    p1 = np.percentile(img,1)
    if p99 is None:
        p99 = np.percentile(img,99.9)
    img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
    masks, flows, styles, diams = model.eval(img, diameter=sz_cell, channels=chan,
                                         flow_threshold=0.85,cellprob_threshold=-2,min_size=sz_min_2d,normalize=False)

### Save psf

In [4]:
psfs = []

In [ ]:
master_folder = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\P2_Sox11_Sox2_Dcx__set2'

for ifov in tqdm([11,12,13,14,15,16,17,18]):
    im = read_im(master_folder+r'\Conv_zscan__'+str(ifov).zfill(3)+'.zarr')
    im_ = np.array(im[0][1:,500:2500,500:2500],dtype=np.float32)
    psf = get_psf(im_,th=1000,th_cor = 0.75,delta=3,delta_fit = 7,sxyzP = [15,60,60])
    psfs.append(psf)
#napari.view_image(im)
psff = np.mean([psf for psf in psfs if psf is not None],axis=0)
psff_ = np.array([p-np.median(p) for p in psff])
from scipy.ndimage import median_filter
psff_med = median_filter(psff_, size=15)
psfff = (psff_-psff_med)[5:-5,5:-5,5:-5][:-1,:-1,:-1]
psfff[psfff<0]=0
psfff = psfff/np.max(psfff)
np.save('psf_750_Scope3_final.npy',psfff)

In [16]:
if False:
    imd = apply_deconv(im_[:,750:-750,750:-750],
        psf=psf,
        plt_val=True,
        parameters={'method': 'wiener', 'beta': 0.0001, 'niter': 50},
        gpu=False,
        force=True,
        pad=None)

In [22]:
import shutil
save_folder = os.path.dirname(master_folder)+os.sep+'MERFISH_Analysis'
if not os.path.exists(save_folder): os.makedirs(save_folder)
shutil.copy2(os.path.abspath('psf_750_Scope3_final.npy'),save_folder+os.sep+'psf_750_Scope3_final.npy')

'\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy'

### Save flat field

In [24]:
#np.load(r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis\med_col0.npz')['im'].shape

(300, 300)

In [27]:
data_fld = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\P2_Sox11_Sox2_Dcx__set1'
save_folder = os.path.dirname(master_folder)+os.sep+'MERFISH_Analysis'
if not os.path.exists(save_folder): os.makedirs(save_folder)

In [ ]:
compute_flat_field_raw(data_fld=data_fld,
    save_folder=save_folder,
    ncols=4)

 94%|███████████████████████████████████████████████████████████████████████████▏    | 656/698 [04:16<00:19,  2.20it/s]